In [ ]:
# First, you need to install the library for the Gemini API.
# Open your terminal or command prompt and run:
# pip install google-generativeai

import google.generativeai as genai
import os # Import the 'os' library to access environment variables
import time # Import the 'time' library to add delays

# --- Configuration ---
# Load the API key from an environment variable for better security.
API_KEY = os.getenv("GEMINI_API_KEY")

if not API_KEY:
    print("Error: GEMINI_API_KEY environment variable not set.")
    print("Please set the environment variable before running the script.")
    exit()

try:
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring API. Please ensure your API key is valid. Error: {e}")
    exit()

# --- UPGRADED: A More Sophisticated Prompt with a Wider Variety of Examples ---
prompt = """
You are an expert financial analyst. Your task is to classify a chunk of text from a company's filing into one of the following categories:
- RISK_TITLE: A short, distinct heading for a specific forward-looking risk.
- RISK_EXPLANATION: The paragraph(s) that explain the risk mentioned in the title.
- FINANCIAL_SUMMARY: A backward-looking discussion of financial performance, revenues, costs, or operations.
- BOILERPLATE: Generic, non-informative text, often legal disclaimers.
- OTHER: Any text that does not fit the above categories, such as a table of contents line or a page number.

Here are some high-quality, diverse examples:

Text: 'WE HAVE GENERATED SIGNIFICANT LOSSES IN RECENT PERIODS.'
Label: RISK_TITLE

Text: 'We incurred significant net losses during the three years prior to 2004.'
Label: RISK_EXPLANATION

Text: 'Our Acquisitions May Not Perform as Anticipated'
Label: RISK_TITLE

Text: 'We have completed many acquisitions of self-storage facilities since our initial public offering of common stock in June 1995.'
Label: RISK_EXPLANATION

Text: 'RESTRICTIVE COVENANTS IN OUR DEBT INSTRUMENTS COULD LIMIT OUR FINANCIAL AND OPERATING FLEXIBILITY AND SUBJECT US TO OTHER RISKS.'
Label: RISK_TITLE

Text: 'The agreements governing our indebtedness include certain covenants that restrict, among other things, our ability to: - incur additional debt; - pay dividends on our common stock in excess of 10 million per year;'
Label: RISK_EXPLANATION

Text: 'Results of Operations'
Label: FINANCIAL_SUMMARY

Text: 'Net income for the year ended December 31, 2004 was 62.1 million compared to 49.1 million for year ended December 31, 2003.'
Label: FINANCIAL_SUMMARY

Text: 'You should carefully consider the risks described below, together with all of the other information included in or incorporated by reference into our Form 10-K, as part of your evaluation of the Company.'
Label: BOILERPLATE

Text: 'Item 1A. Risk Factors'
Label: OTHER

---
Now, classify the following text. Respond with only ONE of the labels: RISK_TITLE, RISK_EXPLANATION, FINANCIAL_SUMMARY, BOILERPLATE, or OTHER.

Text: '{text_to_classify}'
Label: 
"""

def classify_chunk(text_chunk):
    """
    Uses the Gemini API with a multi-class prompt to classify a text chunk.

    Args:
        text_chunk (str): The piece of text to classify.

    Returns:
        str: The predicted label.
    """
    # --- MODEL CHANGE: Using a faster model with a higher rate limit ---
    model = genai.GenerativeModel('gemini-2.5-flash') 
    
    try:
        formatted_prompt = prompt.format(text_to_classify=text_chunk)
        response = model.generate_content(formatted_prompt)
        return response.text.strip()
        
    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        return "API_ERROR"

# --- Example Usage ---
if __name__ == "__main__":
    
    test_chunks = [
        "OUR INVENTORIES ARE NOT MANAGED BY PERPETUAL INVENTORY CONTROL SYSTEMS.", # Should be RISK_TITLE
        "The systems and processes we use to manage and value our inventories require significant manual intervention.", # Should be RISK_EXPLANATION
        "Gross profit of 210 million in 2004 represented a 42 million, or 25 , increase from gross profit of 168 million in 2003.", # Should be FINANCIAL_SUMMARY
        "In addition to the other information in this Form 10-K, the following factors should be considered in evaluating our company and our business.", # Should be BOILERPLATE
        "Supplier Risks", # Should be RISK_TITLE
        "Our success is dependent on the performance of our vendors and service providers.", # Should be RISK_TITLE
    ]

    print("--- Running AI Multi-Class Classification on Test Chunks ---")
    for chunk in test_chunks:
        label = classify_chunk(chunk)
        print(f"\nText:  '{chunk}'")
        print(f"AI Label: -> {label} <-")
        
        # --- BEST PRACTICE: Add a small delay to respect API rate limits ---
        # A 1-second delay allows for up to 60 requests per minute.
        time.sleep(1)



--- Running AI Multi-Class Classification on Test Chunks ---

Text:  'OUR INVENTORIES ARE NOT MANAGED BY PERPETUAL INVENTORY CONTROL SYSTEMS.'
AI Label: -> RISK_TITLE <-

Text:  'The systems and processes we use to manage and value our inventories require significant manual intervention.'
AI Label: -> RISK_EXPLANATION <-

Text:  'Gross profit of 210 million in 2004 represented a 42 million, or 25 , increase from gross profit of 168 million in 2003.'
AI Label: -> FINANCIAL_SUMMARY <-

Text:  'In addition to the other information in this Form 10-K, the following factors should be considered in evaluating our company and our business.'
AI Label: -> BOILERPLATE <-

Text:  'Supplier Risks'
AI Label: -> RISK_TITLE <-

Text:  'Our success is dependent on the performance of our vendors and service providers.'
AI Label: -> RISK_TITLE <-


In [2]:
# This script combines file loading, AI classification, and intelligent structuring.
import google.generativeai as genai
import os
import time
import glob
import pickle
import pandas as pd
import re
import json

# --- Configuration & API Setup ---
API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    print("Error: GEMINI_API_KEY environment variable not set.")
    exit()
try:
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring API: {e}")
    exit()

# --- UPGRADED: AI Prompt with instructions for handling lists ---
prompt = """
You are an expert financial analyst. Your task is to classify a chunk of text from a company's filing into one of the following categories:
- RISK_TITLE: A short, distinct heading for a specific forward-looking risk.
- RISK_EXPLANATION: The paragraph(s) that explain the risk mentioned in the title. This INCLUDES bullet points or lists that follow an introductory sentence.
- FINANCIAL_SUMMARY: A backward-looking discussion of financial performance, revenues, costs, or operations.
- BOILERPLATE: Generic, non-informative text, often legal disclaimers.
- OTHER: Any text that does not fit the above categories, such as a table of contents line or a page number.

Here are some high-quality, diverse examples:

Text: 'WE HAVE GENERATED SIGNIFICANT LOSSES IN RECENT PERIODS.'
Label: RISK_TITLE

Text: 'We incurred significant net losses during the three years prior to 2004.'
Label: RISK_EXPLANATION

Text: 'Our Acquisitions May Not Perform as A nticipated'
Label: RISK_TITLE

Text: 'We have completed many acquisitions of self-storage facilities since our initial public offering of common stock in June 1995.'
Label: RISK_EXPLANATION

Text: 'The agreements governing our indebtedness include certain covenants that restrict, among other things, our ability to:'
Label: RISK_EXPLANATION

Text: '- incur additional debt; - pay dividends on our common stock in excess of 10 million per year; - repurchase our common stock;'
Label: RISK_EXPLANATION

Text: 'Results of Operations'
Label: FINANCIAL_SUMMARY

Text: 'Net income for the year ended December 31, 2004 was 62.1 million compared to 49.1 million for year ended December 31, 2003.'
Label: FINANCIAL_SUMMARY

Text: 'You should carefully consider the risks described below, together with all of the other information included in or incorporated by reference into our Form 10-K, as part of your evaluation of the Company.'
Label: BOILERPLATE

Text: 'Item 1A. Risk Factors'
Label: OTHER
---
Now, classify the following text. Respond with only ONE of the labels.
Text: '{text_to_classify}'
Label: 
"""

def classify_chunk_with_retry(text_chunk, max_retries=3, delay=5):
    """Uses the Gemini API to classify a text chunk, with retry logic for rate limits."""
    model = genai.GenerativeModel('gemini-2.5-flash')
    for attempt in range(max_retries):
        try:
            formatted_prompt = prompt.format(text_to_classify=text_chunk)
            response = model.generate_content(formatted_prompt)
            # A small delay to help stay under the rate limit
            time.sleep(2) 
            return response.text.strip()
        except Exception as e:
            print(f"  > API Error: {e}. Retrying in {delay}s... (Attempt {attempt + 1}/{max_retries})")
            time.sleep(delay)
    return "API_FAILURE"

def process_filing_to_json(filing_series, output_filename="structured_risk_factors.json"):
    """
    Takes a single filing (as a pandas Series), processes its risk factors using AI,
    structures the result, and saves it as a JSON file.
    """
    raw_text = filing_series.get('rf', '')
    if not raw_text or not isinstance(raw_text, str):
        print("No valid risk factor text found in this filing.")
        return

    # 1. Split the raw text into processable chunks (paragraphs)
    chunks = re.split(r'\n\s*\n', raw_text.strip())
    cleaned_chunks = [ ' '.join(c.split()) for c in chunks if c.strip() and not c.strip().isdigit() ]

    # 2. Get AI-powered labels for each chunk
    print(f"\nClassifying {len(cleaned_chunks)} text chunks with Gemini API...")
    labeled_chunks = []
    for i, chunk in enumerate(cleaned_chunks):
        label = classify_chunk_with_retry(chunk)
        print(f"  Chunk {i+1}/{len(cleaned_chunks)} classified as: {label}")
        labeled_chunks.append({'text': chunk, 'label': label})

    # 3. Structure the data by "fusing" explanations to their titles
    print("\nStructuring the labeled data...")
    structured_risks = []
    current_risk = None
    for item in labeled_chunks:
        label = item['label']
        text = item['text']

        if label == 'RISK_TITLE':
            # If a new title is found, the previous risk is complete.
            if current_risk:
                structured_risks.append(current_risk)
            # Start a new risk item.
            current_risk = {'title': text, 'explanation': []}
        
        elif label == 'RISK_EXPLANATION':
            # If this is an explanation, append it to the current risk item.
            if current_risk:
                current_risk['explanation'].append(text)
            # If an explanation appears before any title, create a risk item without a title.
            else:
                current_risk = {'title': 'Uncategorized', 'explanation': [text]}


    # After the loop, add the last risk item that was being built.
    if current_risk:
        structured_risks.append(current_risk)

    # Join the list of explanation paragraphs into a single string for each risk
    for risk in structured_risks:
        risk['explanation'] = ' '.join(risk['explanation'])

    # 4. Save the final structured data to a JSON file
    final_output = {
        'cik': str(filing_series.get('cik', 'N/A')),
        'date': str(filing_series.get('date', 'N/A')),
        'structured_risks': structured_risks
    }
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, indent=4)
        
    print(f"\nSuccess! Structured risk data has been saved to '{output_filename}'")
    print("You can now open this JSON file to see the clean, machine-readable output.")

# --- Main Execution Block ---
if __name__ == "__main__":
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    
    # Find and load the first pickle file
    pickle_files = glob.glob(os.path.join(text_data_path, '**', '*.pkl'), recursive=True)
    if not pickle_files:
        print("No pickle files found.")
        exit()
        
    print(f"Loading data from: {os.path.basename(pickle_files[0])}")
    with open(pickle_files[0], 'rb') as f:
        filings_df = pickle.load(f)

    # Find the first valid filing to process
    valid_filing = None
    for _, row in filings_df.iterrows():
        if pd.notna(row.get('rf')) and len(str(row.get('rf')).strip()) > 100:
            valid_filing = row
            break
            
    if valid_filing is not None:
        process_filing_to_json(valid_filing)
    else:
        print("Could not find a valid filing with risk factors in the first pickle file.")



Loading data from: text_us_2005.pkl


C:\Users\carte\AppData\Local\Temp\ipykernel_29780\3112969323.py:162: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  filings_df = pickle.load(f)



Classifying 57 text chunks with Gemini API...
  Chunk 1/57 classified as: OTHER
  Chunk 2/57 classified as: BOILERPLATE
  Chunk 3/57 classified as: OTHER
  Chunk 4/57 classified as: RISK_TITLE
  Chunk 5/57 classified as: RISK_EXPLANATION
  Chunk 6/57 classified as: RISK_TITLE
  Chunk 7/57 classified as: RISK_EXPLANATION
  Chunk 8/57 classified as: RISK_TITLE
  Chunk 9/57 classified as: RISK_EXPLANATION
  Chunk 10/57 classified as: RISK_TITLE
  Chunk 11/57 classified as: RISK_EXPLANATION
  > API Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_v

"""
You are an expert financial analyst. Your task is to classify a chunk of text from a company's filing into one of the following categories:
- RISK_TITLE: A short, distinct heading for a specific forward-looking risk.
- RISK_EXPLANATION: The paragraph(s) that explain the risk mentioned in the title. This INCLUDES bullet points or lists that follow an introductory sentence.
- FINANCIAL_SUMMARY: A backward-looking discussion of financial performance, revenues, costs, or operations.
- BOILERPLATE: Generic, non-informative text, often legal disclaimers.
- OTHER: Any text that does not fit the above categories, such as a table of contents line or a page number.

Here are some high-quality, diverse examples:

Text: 'WE HAVE GENERATED SIGNIFICANT LOSSES IN RECENT PERIODS.'
Label: RISK_TITLE

Text: 'We incurred significant net losses during the three years prior to 2004.'
Label: RISK_EXPLANATION

Text: 'Our Acquisitions May Not Perform as Anticipated'
Label: RISK_TITLE

Text: 'Our success depends on our ability to respond to changes in customer demands and fashion trends in a timely manner.'
Label: RISK_TITLE

Text: 'We have completed many acquisitions of self-storage facilities since our initial public offering of common stock in June 1995.'
Label: RISK_EXPLANATION

Text: 'The agreements governing our indebtedness include certain covenants that restrict, among other things, our ability to:'
Label: RISK_EXPLANATION

Text: '- incur additional debt; - pay dividends on our common stock in excess of 10 million per year; - repurchase our common stock;'
Label: RISK_EXPLANATION

Text: 'Results of Operations'
Label: FINANCIAL_SUMMARY

Text: 'Net income for the year ended December 31, 2004 was 62.1 million compared to 49.1 million for year ended December 31, 2003.'
Label: FINANCIAL_SUMMARY

Text: 'You should carefully consider the risks described below, together with all of the other information included in or incorporated by reference into our Form 10-K, as part of your evaluation of the Company.'
Label: BOILERPLATE

Text: 'Item 1A. Risk Factors'
Label: OTHER
---
Now, classify the following text. Respond with only ONE of the labels.
Text: '{text_to_classify}'
Label: 
"""

In [4]:
# This script combines file loading, AI classification, and intelligent structuring.
import google.generativeai as genai
import os
import time
import glob
import pickle
import pandas as pd
import re
import json

# --- Configuration & API Setup ---
API_KEY = os.getenv("GEMINI_API_KEY")
if not API_KEY:
    print("Error: GEMINI_API_KEY environment variable not set.")
    exit()
try:
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring API: {e}")
    exit()

# --- UPGRADED: AI Prompt with instructions for handling lists ---
prompt = """
You are an expert financial analyst. Your task is to classify a chunk of text from a company's filing into one of the following categories:
- RISK_TITLE: A short, distinct heading for a specific forward-looking risk.
- RISK_EXPLANATION: The paragraph(s) that explain the risk mentioned in the title. This INCLUDES bullet points or lists that follow an introductory sentence.
- FINANCIAL_SUMMARY: A backward-looking discussion of financial performance, revenues, costs, or operations.
- BOILERPLATE: Generic, non-informative text, often legal disclaimers.
- OTHER: Any text that does not fit the above categories, such as a table of contents line or a page number.

Here are some high-quality, diverse examples:

Text: 'WE HAVE GENERATED SIGNIFICANT LOSSES IN RECENT PERIODS.'
Label: RISK_TITLE

Text: 'We incurred significant net losses during the three years prior to 2004.'
Label: RISK_EXPLANATION

Text: 'Our Acquisitions May Not Perform as Anticipated'
Label: RISK_TITLE

Text: 'Our success depends on our ability to respond to changes in customer demands and fashion trends in a timely manner.'
Label: RISK_TITLE

Text: 'We have completed many acquisitions of self-storage facilities since our initial public offering of common stock in June 1995.'
Label: RISK_EXPLANATION

Text: 'The agreements governing our indebtedness include certain covenants that restrict, among other things, our ability to:'
Label: RISK_EXPLANATION

Text: '- incur additional debt; - pay dividends on our common stock in excess of 10 million per year; - repurchase our common stock;'
Label: RISK_EXPLANATION

Text: 'Results of Operations'
Label: FINANCIAL_SUMMARY

Text: 'Net income for the year ended December 31, 2004 was 62.1 million compared to 49.1 million for year ended December 31, 2003.'
Label: FINANCIAL_SUMMARY

Text: 'You should carefully consider the risks described below, together with all of the other information included in or incorporated by reference into our Form 10-K, as part of your evaluation of the Company.'
Label: BOILERPLATE

Text: 'Item 1A. Risk Factors'
Label: OTHER
---
Now, classify the following text. Respond with only ONE of the labels.
Text: '{text_to_classify}'
Label: 
"""

def classify_chunk_with_retry(text_chunk, max_retries=5, delay=8):
    """Uses the Gemini API to classify a text chunk, with more conservative retry logic."""
    model = genai.GenerativeModel('gemini-2.5-flash')
    for attempt in range(max_retries):
        try:
            formatted_prompt = prompt.format(text_to_classify=text_chunk)
            response = model.generate_content(formatted_prompt)
            # A longer delay to be more respectful of the API rate limits.
            time.sleep(5) 
            return response.text.strip()
        except Exception as e:
            print(f"  > API Error: {e}. Retrying in {delay}s... (Attempt {attempt + 1}/{max_retries})")
            time.sleep(delay)
    return "API_FAILURE"

def process_filing_to_json(filing_series, output_dir, overwrite_existing=False):
    """
    Takes a single filing, processes its risk factors using AI,
    and saves the structured result to a uniquely named JSON file in the output directory.
    """
    raw_text = filing_series.get('rf', '')
    cik = str(filing_series.get('cik', 'unknown_cik'))
    date = str(filing_series.get('date', 'unknown_date'))
    output_filename = f"{cik}_{date}.json"
    output_path = os.path.join(output_dir, output_filename)

    # Skip if the file already exists, UNLESS we are in overwrite mode.
    if not overwrite_existing and os.path.exists(output_path):
        print(f"  > Skipping, output file already exists: {output_filename}")
        return

    if not raw_text or not isinstance(raw_text, str) or len(raw_text.strip()) < 100:
        print("  > Skipping, no valid risk factor text found.")
        return

    # 1. Split the raw text into processable chunks (paragraphs)
    chunks = re.split(r'\n\s*\n', raw_text.strip())
    cleaned_chunks = [ ' '.join(c.split()) for c in chunks if c.strip() and not c.strip().isdigit() ]

    # 2. Get AI-powered labels for each chunk
    print(f"  > Classifying {len(cleaned_chunks)} text chunks with Gemini API...")
    labeled_chunks = []
    for i, chunk in enumerate(cleaned_chunks):
        label = classify_chunk_with_retry(chunk)
        labeled_chunks.append({'text': chunk, 'label': label})

    # 3. Structure the data by "fusing" explanations to their titles
    structured_risks = []
    current_risk = None
    for item in labeled_chunks:
        label = item['label']
        text = item['text']

        if label == 'RISK_TITLE':
            if current_risk:
                structured_risks.append(current_risk)
            current_risk = {'title': text, 'explanation': []}
        
        elif label == 'RISK_EXPLANATION':
            if current_risk:
                current_risk['explanation'].append(text)
            else:
                current_risk = {'title': 'Uncategorized', 'explanation': [text]}

    if current_risk:
        structured_risks.append(current_risk)

    for risk in structured_risks:
        risk['explanation'] = ' '.join(risk['explanation'])

    # 4. Save the final structured data to a JSON file
    final_output = {
        'cik': cik,
        'date': date,
        'structured_risks': structured_risks
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, indent=4)
        
    print(f"  > Success! Structured data saved to '{output_filename}'")

# --- Main Execution Block ---
if __name__ == "__main__":
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    output_dir = "structured_json_outputs"
    
    START_FROM_PICKLE_FILE = None 
    
    OVERWRITE_STARTING_FILE_ONLY = False
    
    OVERWRITE_EXISTING_FILES = True

    print(f"Output will be saved to the '{output_dir}' directory.")
    os.makedirs(output_dir, exist_ok=True)
    
    pickle_files = sorted(glob.glob(os.path.join(text_data_path, '**', '*.pkl'), recursive=True))
    if not pickle_files:
        print("No pickle files found.")
        exit()

    start_processing = START_FROM_PICKLE_FILE is None
        
    for pkl_file_path in pickle_files:
        current_file_name = os.path.basename(pkl_file_path)

        if not start_processing and current_file_name == START_FROM_PICKLE_FILE:
            print(f"--- Found start file '{current_file_name}'. Resuming processing. ---")
            start_processing = True

        if not start_processing:
            print(f"Skipping file: {current_file_name} (waiting for start file)")
            continue

        print(f"\n--- Loading Pickle File: {current_file_name} ---")
        try:
            with open(pkl_file_path, 'rb') as f:
                filings_df = pickle.load(f)

            if filings_df.empty:
                print("  > DataFrame is empty, skipping file.")
                continue
            
            should_overwrite_this_file = OVERWRITE_EXISTING_FILES or \
                                       (OVERWRITE_STARTING_FILE_ONLY and current_file_name == START_FROM_PICKLE_FILE)
            
            if should_overwrite_this_file:
                 print(f"  > NOTE: Overwrite mode is ON for this file.")

            for index, row in filings_df.iterrows():
                 print(f"\nProcessing filing index {index} from {current_file_name}...")
                 process_filing_to_json(row, output_dir, should_overwrite_this_file)

        except Exception as e:
            print(f"An error occurred processing {current_file_name}: {e}")



Output will be saved to the 'structured_json_outputs' directory.

--- Loading Pickle File: text_us_2005.pkl ---


C:\Users\carte\AppData\Local\Temp\ipykernel_29780\575247828.py:188: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  filings_df = pickle.load(f)


  > NOTE: Overwrite mode is ON for this file.

Processing filing index 177795 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing index 177791 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing index 177794 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing index 177790 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing index 177793 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing index 177798 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing index 177800 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing index 177796 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing index 177799 from text_us_2005.pkl...
  > Skipping, no valid risk factor text found.

Processing filing

KeyboardInterrupt: 

In [1]:
import zipfile
import os
import glob

BASE_DIR = r'C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr'

zip_files = glob.glob(os.path.join(BASE_DIR, '**', '*.zip'), recursive=True)

for file in zip_files:

    extract_to = os.path.join(BASE_DIR, os.path.splitext(os.path.basename(file))[0])
    os.makedirs(extract_to, exist_ok=True)

    with zipfile.ZipFile(file, 'r') as z:
        z.extractall(extract_to)

    print(f"Extracted: {file} → {extract_to}")

Extracted: C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2005.zip → C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2005
Extracted: C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2006.zip → C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2006
Extracted: C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2007.zip → C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2007
Extracted: C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2008.zip → C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2008
Extracted: C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2009.zip → C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2009
Extracted: C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2010.zip → C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2010
Extracted: C:\_Files\School\Competitions\FIAM2025\data\us_text_data_yr\2011.zip → C:\_Files\School\Competitions\FIAM20

In [3]:
import pandas as pd
import os

# Define the source CSV path
csv_path = r'C:\_Files\School\Competitions\FIAM2025\data\quant_data\ret_sample.csv'

# Define a new output path for the Parquet file
# (e.g., 'ret_sample.parquet' in the same directory)
base_path = os.path.splitext(csv_path)[0]
parquet_path = base_path + '.parquet'

# print(f"Input: {csv_path}")
# print(f"Output: {parquet_path}")

try:
    # Read the CSV.
    # Force the 'iid' column to be read as a string ('str')
    # This correctly handles NaNs and numeric-looking IDs.
    df = pd.read_csv(csv_path, dtype={'iid': str})

    # You can also set low_memory=False to silence the DtypeWarning,
    # but specifying dtypes is the better fix.
    # df = pd.read_csv(csv_path, dtype={'iid': str}, low_memory=False)

    # Save the DataFrame to the new PARQUET path
    df.to_parquet(parquet_path)
    
    print(f"Successfully converted to {parquet_path}")

except Exception as e:
    print(f"An error occurred: {e}")
    # If this still fails, you may need to investigate column 4
    # Find its name and add it to the dtype dictionary as well, e.g.:
    # df = pd.read_csv(csv_path, dtype={'iid': str, 'name_of_col_4': str})

Successfully converted to C:\_Files\School\Competitions\FIAM2025\data\quant_data\ret_sample.parquet
